In [ ]:
# import the libraries
import csv
import cv2
import numpy as np
print(cv2.__version__)

In [4]:
current_dir = "./180204-00/" #"./180203-00/"
model_name = "180209-00.h5"
IMG_SHAPE = (160, 320, 3)
lines = [] 
images = []
measurements = []

In [ ]:
def add_training_data(directory):
    csv_fname = directory + "driving_log.csv"
    print(csv_fname)
    f_csv = open (csv_fname)
    print(f_csv)
    read_buffer = csv.reader(f_csv)
    for a_line in read_buffer:
        lines.append(a_line)

    #modify source path to relative path
    for a_line in lines:
        orig_source_path = a_line [0]  #center image

        #exclude any header lines
        if (a_line[0] == "center"):
            print(a_line)
            continue

        f_name = orig_source_path.split('\\')[-1]  # just the file name portion for windows based files
        current_image_path = directory +"IMG/" + f_name
        #print(current_image_path)
        an_img  = cv2.imread(current_image_path, cv2.COLOR_BGR2RGB)
        if (an_img.all() == None):
            print("None file: ",current_image_path)
            continue
            
        #add the image & steering to the training set    
        images.append(an_img)
        measurements.append (float(a_line[3]))
        
        #add the flip side of the image
        #image_flipped = np.fliplr(image)
        #measurement_flipped = -measurement
        images.append(np.fliplr(an_img))
        measurements.append (-float(a_line[3]))
        
        #close the opened file
        f_csv.close()

In [ ]:
add_training_data(current_dir)
#add_training_data("./180203-00/")

X_train = np.array(images)
y_train = np.array(measurements)
    
print("X_train shape {}, y_train.shape".format(X_train.shape, y_train.shape))

In [ ]:
print(images[0])

In [5]:
N_EPOCHS = 10

In [6]:
#Create and train the model.
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Lambda
from keras.layers import Convolution2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense

from keras.callbacks import ModelCheckpoint

In [12]:
### Define your architecture here.
### TODO: Define your architecture.
n_filters1 = 24
krnl_sz1 = 5                ## kernel size for convolutions - smaller better for better localization.
padding1 = 'valid'
activation1 = 'relu'
dropout1 = 0.1
pool_sz1 = 2

n_filters2 = 36
krnl_sz2 = 5                ## kernel size for convolutions - smaller better for better localization.
padding2 = 'valid'
activation2 = 'relu'
dropout2 = 0.1

n_filters2 = 48
krnl_sz2 = 5                ## kernel size for convolutions - smaller better for better localization.
padding2 = 'valid'
activation2 = 'relu'
dropout2 = 0.1
pool_sz2 = 2

n_filters3 = 64
krnl_sz3 = 5                ## kernel size for convolutions - smaller better for better localization.
padding3 = 'valid'
activation3 = 'relu'
dropout3 = 0.1
pool_sz3 = 2

n_nodes4 = 64
activation4 = 'relu'
dropout4 = 0.1

n_nodes5 = 32
activation5 = 'relu'
dropout5 = 0.1

n_nodes6 = 16
activation6 = 'relu'
dropout6 = 0.1

n_classes= 1             ## output classes - nornalized  x, y positions of each feature point x 15


#start with input layer with normalization of mean = 0 and range [-0.5: 0.5]
model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape= IMG_SHAPE))


#Take NVIDIA paper suggested architecture as per 
# https://devblogs.nvidia.com/deep-learning-self-driving-cars/


## Layer 1
##  Define the first 2D convolutional layer with proper input shape and 32 filters to start with, relu activation.
model.add(Convolution2D(n_filters1,krnl_sz1 ,krnl_sz1, border_mode=padding1, activation = activation1))
model.add(MaxPooling2D(pool_size= (pool_sz1, pool_sz1))) ## Add pooling layer
model.add(Dropout(dropout1))    ## dropout reduces the risk of overfitting.


## Layer 2
model.add(Convolution2D(n_filters2,krnl_sz2 ,krnl_sz2, border_mode=padding2, activation = activation2))
model.add(MaxPooling2D(pool_size= (pool_sz2, pool_sz2)))
model.add(Dropout(dropout2))    ## dropout reduces the risk of overfitting.

## Layer 3
model.add(Convolution2D(n_filters3,krnl_sz3 ,krnl_sz3, border_mode=padding3, activation = activation3))
model.add(MaxPooling2D(pool_size= (pool_sz3, pool_sz3)))
model.add(Dropout(dropout3))    ## dropout reduces the risk of overfitting.


# Flatten => RELU layers
model.add(Flatten())

### Layer 4
model.add(Dense(n_nodes4, activation= activation4))
model.add(Dropout(dropout4)) ## Add strong drop out rate at this later stage.

### Layer 5
model.add(Dense(n_nodes5, activation= activation5))
model.add(Dropout(dropout5)) ## Add strong drop out rate at this later stage.

### Layer 6
model.add(Dense(n_nodes6, activation= activation6))
model.add(Dropout(dropout6)) ## Add strong drop out rate at this later stage.

### Layer 7
model.add(Dense(n_classes))

# Summarize the model
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_4 (Lambda)                (None, 160, 320, 3)   0           lambda_input_4[0][0]             
____________________________________________________________________________________________________
convolution2d_4 (Convolution2D)  (None, 156, 316, 24)  1824        lambda_4[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 78, 158, 24)   0           convolution2d_4[0][0]            
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 78, 158, 24)   0           maxpooling2d_4[0][0]             
___________________________________________________________________________________________

In [ ]:
## TODO: Compile the model
model.compile(loss ='mse',optimizer='adam')
# TODO: Compile the model using a loss function and an optimizer.
#model.compile(loss = 'mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])

## TODO: Train the model
checkpointer = ModelCheckpoint(filepath='weights.best.adam.hdf5', 
                               verbose=1, save_best_only=True)

# TODO: Run the model. Feel free to experiment with different batch sizes and number of epochs.
hist = model.fit(X_train, y_train, nb_epoch = N_EPOCHS, shuffle = True, callbacks=[checkpointer], verbose=1, validation_split=0.2)

#model.fit(X_train, y_train, validation_split = 0.2, shuffle = True, nb_epoch = N_EPOCHS)

#save model
model.save (model_name)
